In [2]:
import tensorflow as tf
import numpy as np
import cv2
import os
import keras
from keras.models import Sequential, model_from_json
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from keras.utils import to_categorical
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

C:\Users\Bhumit\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
# Set dir path and get images
path = os.getcwd()
train = path + '/data'
train_dir_list = os.listdir(train)
train_dir_list

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'a',
 'b',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'l',
 'm',
 'n',
 'q',
 'r',
 't',
 'y',
 '_A',
 '_B',
 '_C',
 '_D',
 '_E',
 '_F',
 '_G',
 '_H',
 '_I',
 '_J',
 '_K',
 '_L',
 '_M',
 '_N',
 '_O',
 '_P',
 '_Q',
 '_R',
 '_S',
 '_T',
 '_U',
 '_V',
 '_W',
 '_X',
 '_Y',
 '_Z']

In [ ]:
train_dir_list[31]

In [ ]:
train_images_list = []
label = 0
labels = []

for folder in train_dir_list:
    img_list = os.listdir(train + '/' + folder)
    print('Working Folder:',folder)
    for img in img_list:
        ip_img = cv2.imread(train + '/' + folder + '/' + img, 0)
        ip_img = cv2.resize(ip_img, (28,28))
        labels.append(label)
        train_images_list.append(ip_img)
    label += 1

In [ ]:
# Convert to array
train = np.array(train_images_list).astype(np.float32)
labels = np.array(labels)
train /= 255
#Convert to categorical
labels = to_categorical(labels, 52)
train = train.reshape((train.shape[0], 28, 28, 1))
train.shape

In [ ]:
np.save('train_data.npy', train)
np.save('labels.npy', labels)

In [5]:
x, y = shuffle(np.load('train_data.npy'), np.load('labels.npy'), random_state=5)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [4]:
# Now create our model
tf.reset_default_graph()

'''model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(52, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])'''

model = Sequential()
model.add(Conv2D(32, (3,3), strides=(1,1),activation='relu', input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Conv2D(64 ,(3,3),strides=(1,1), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(784, activation='relu'))
model.add(Dense(52, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 128)         73856     
__________

In [8]:
history = model.fit(x_train, y_train, batch_size=500, epochs=5, verbose=1, validation_data=(x_test,y_test))

Train on 729472 samples, validate on 312632 samples
Epoch 1/5
729472/729472 [==============================] - 335s 460us/step - loss: 0.3596 - acc: 0.8684 - val_loss: 0.2669 - val_acc: 0.8975
Epoch 2/5
729472/729472 [==============================] - 338s 463us/step - loss: 0.3548 - acc: 0.8701 - val_loss: 0.2664 - val_acc: 0.8992
Epoch 3/5
729472/729472 [==============================] - 337s 462us/step - loss: 0.3498 - acc: 0.8719 - val_loss: 0.2641 - val_acc: 0.9009
Epoch 4/5
729472/729472 [==============================] - 342s 468us/step - loss: 0.3465 - acc: 0.8730 - val_loss: 0.2598 - val_acc: 0.9025
Epoch 5/5
729472/729472 [==============================] - 339s 465us/step - loss: 0.3432 - acc: 0.8741 - val_loss: 0.2603 - val_acc: 0.9021


In [9]:
# Save model to json
j_model = model.to_json()
with open('ocr14_model2_epochs15_acc90-.json', 'w') as f:
    f.write(j_model)
model.save_weights('ocr14_model2_epochs15_acc90-.h5')

In [10]:
history = model.fit(x_train, y_train, batch_size=500, epochs=5, verbose=1, validation_data=(x_test,y_test))

Train on 729472 samples, validate on 312632 samples
Epoch 1/5
729472/729472 [==============================] - 333s 456us/step - loss: 0.3409 - acc: 0.8751 - val_loss: 0.2576 - val_acc: 0.9032
Epoch 2/5
729472/729472 [==============================] - 335s 460us/step - loss: 0.3379 - acc: 0.8757 - val_loss: 0.2634 - val_acc: 0.9010
Epoch 3/5
729472/729472 [==============================] - 335s 459us/step - loss: 0.3357 - acc: 0.8766 - val_loss: 0.2579 - val_acc: 0.9026
Epoch 4/5
729472/729472 [==============================] - 335s 460us/step - loss: 0.3337 - acc: 0.8768 - val_loss: 0.2554 - val_acc: 0.9038
Epoch 5/5
729472/729472 [==============================] - 333s 457us/step - loss: 0.3316 - acc: 0.8779 - val_loss: 0.2535 - val_acc: 0.9040


In [11]:
# Save model to json
j_model = model.to_json()
with open('ocr14_model2_epochs20_acc90-.json', 'w') as f:
    f.write(j_model)
model.save_weights('ocr14_model2_epochs20_acc90-.h5')

In [7]:
# Load model
# Load model
json_model = open('ocr14_model2_epochs20_acc90-.json')
ld_model = json_model.read()
json_model.close()
model = model_from_json(ld_model)
model.load_weights('ocr14_model2_epochs20_acc90-.h5')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 128)         73856     
__________

In [8]:
#25th epochs result
history = model.fit(x_train, y_train, batch_size=500, epochs=5, verbose=1, validation_data=(x_test,y_test))

Train on 729472 samples, validate on 312632 samples
Epoch 1/5
729472/729472 [==============================] - 330s 453us/step - loss: 0.3373 - acc: 0.8766 - val_loss: 0.2367 - val_acc: 0.9093
Epoch 2/5
729472/729472 [==============================] - 329s 450us/step - loss: 0.3320 - acc: 0.8777 - val_loss: 0.2425 - val_acc: 0.9065
Epoch 3/5
729472/729472 [==============================] - 328s 450us/step - loss: 0.3304 - acc: 0.8785 - val_loss: 0.2395 - val_acc: 0.9080
Epoch 4/5
729472/729472 [==============================] - 329s 450us/step - loss: 0.3293 - acc: 0.8782 - val_loss: 0.2402 - val_acc: 0.9085
Epoch 5/5
729472/729472 [==============================] - 330s 452us/step - loss: 0.3257 - acc: 0.8798 - val_loss: 0.2415 - val_acc: 0.9074


In [19]:
# Save model to json
j_model = model.to_json()
with open('ocr14_model2_epochs25_acc90-.json', 'w') as f:
    f.write(j_model)
model.save_weights('ocr14_model2_epochs25_acc90-.h5')

In [20]:
#35th epochs result
history = model.fit(x_train, y_train, batch_size=500, epochs=10, verbose=1, validation_data=(x_test,y_test))

Train on 729472 samples, validate on 312632 samples
Epoch 1/10
729472/729472 [==============================] - 327s 448us/step - loss: 0.3258 - acc: 0.8796 - val_loss: 0.2443 - val_acc: 0.9069
Epoch 2/10
729472/729472 [==============================] - 327s 448us/step - loss: 0.3228 - acc: 0.8808 - val_loss: 0.2401 - val_acc: 0.9084
Epoch 3/10
729472/729472 [==============================] - 326s 447us/step - loss: 0.3221 - acc: 0.8809 - val_loss: 0.2388 - val_acc: 0.9093
Epoch 4/10
729472/729472 [==============================] - 327s 448us/step - loss: 0.3218 - acc: 0.8804 - val_loss: 0.2421 - val_acc: 0.9075
Epoch 5/10
729472/729472 [==============================] - 327s 448us/step - loss: 0.3200 - acc: 0.8812 - val_loss: 0.2369 - val_acc: 0.9097
Epoch 6/10
729472/729472 [==============================] - 338s 464us/step - loss: 0.3194 - acc: 0.8815 - val_loss: 0.2404 - val_acc: 0.9082
Epoch 7/10
729472/729472 [==============================] - 338s 463us/step - loss: 0.3189 - acc

In [21]:
# Save model to json
j_model = model.to_json()
with open('ocr14_model2_epochs35_acc90.json', 'w') as f:
    f.write(j_model)
model.save_weights('ocr14_model2_epochs35_acc90.h5')

In [23]:
img = cv2.imread('4_test.png', 0)
img = np.array(img).astype('float32')
img /= 255
model.predict_classes(img.reshape((1,28,28,1)))

array([46], dtype=int64)